In [1]:
#Import modules
%matplotlib inline

#handling data
import pandas as pd
import numpy as np

#import twitter credentials
from twitter_credentials import *

#handling information
import re
import json

#handling plots
import matplotlib.pyplot as plt

#for network creation
import networkx as nx

In [30]:
tweets_df = pd.read_json("tweets.txt", lines=True)

In [31]:
tweets_df = tweets_df.drop(["contributors", "geo", "coordinates", "display_text_range", "entities", "extended_entities", 
                            "extended_tweet", "favorite_count", "favorited", "filter_level", "id", "in_reply_to_status_id", "in_reply_to_user_id",
                           "timestamp_ms", "truncated", "possibly_sensitive", "quote_count", "quoted_status", "quoted_status_id", 
                            "quoted_status_id_str", "quoted_status_permalink"], axis=1)

In [32]:
tweets_df.columns

Index(['created_at', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
       'is_quote_status', 'lang', 'place', 'reply_count', 'retweet_count',
       'retweeted', 'retweeted_status', 'source', 'text', 'user'],
      dtype='object')

In [34]:
tweets_df.head()

,created_at,id_str,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,is_quote_status,lang,place,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,2019-03-30 20:14:37,1112085741683638272,None,NaN,NaN,True,en,None,0,0,False,{'created_at': 'Sat Mar 30 12:25:59 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @4fnan_zainal: Pls ban kids masa showtime a...,"{'id': 925075656127365120, 'id_str': '92507565..."
1,2019-03-30 20:14:39,1112085751267762176,None,NaN,NaN,False,en,None,0,0,False,{'created_at': 'Sat Mar 30 00:44:39 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @skipbolden: The Avengers (TV Series 1961–1...,"{'id': 812752256214925312, 'id_str': '81275225..."
2,2019-03-30 20:14:41,1112085758851059712,None,NaN,NaN,False,en,None,0,0,False,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",Looking forward to the Avengers End Game 😍 #av...,"{'id': 924977392535339010, 'id_str': '92497739..."
3,2019-03-30 20:14:41,1112085761569054720,None,NaN,NaN,False,en,None,0,0,False,{'created_at': 'Sat Mar 30 17:15:19 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @Hypable: 'Avengers: Endgame' posters have ...,"{'id': 1377394638, 'id_str': '1377394638', 'na..."
4,2019-03-30 20:14:41,1112085762030358528,MTV_iwan,1.112086e+18,8.525729e+17,False,en,None,0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@MTV_iwan @theflairqueenx @BeckyLynchWWE I gue...,"{'id': 1105100693147394048, 'id_str': '1105100..."
